# 1. 

In [1]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn import metrics
from sklearn import preprocessing
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
dataset = pd.read_csv("dermatology.data")
dataset

,erythema,scaling,definite borders,itching,koebner phenomenon,polygonal papules,follicular papules,oral mucosal involvement,knee and elbow involvement,scalp involvement,...,munro microabcess,focal hypergranulosis,disappearance of the granular layer,vacuolisation and damage of basal layer,spongiosis,saw-tooth appearance of retes,follicular horn plug,perifollicular parakeratosis,inflammatory monoluclear inflitrate,band-like infiltrate
0,2,2,0,3,0,0,0,0,1,0,...,0,0,3,0,0,0,1,0,55,2
1,3,3,3,2,1,0,0,0,1,1,...,0,0,0,0,0,0,1,0,8,1
2,2,1,2,3,1,3,0,3,0,0,...,0,2,3,2,0,0,2,3,26,3
3,2,2,2,0,0,0,0,0,3,2,...,3,0,0,0,0,0,3,0,40,1
4,2,3,2,2,2,2,0,2,0,0,...,2,3,2,3,0,0,2,3,45,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
353,2,1,1,0,1,0,0,0,0,0,...,0,0,1,0,0,0,2,0,25,4
354,3,2,1,0,1,0,0,0,0,0,...,1,0,1,0,0,0,2,0,36,4
355,3,2,2,2,3,2,0,2,0,0,...,0,3,0,3,0,0,2,3,28,3
356,2,1,3,1,2,3,0,2,0,0,...,0,2,0,1,0,0,2,3,50,3


In [42]:
target_feature='spongiosis'
data = dataset.drop([target_feature], axis=1)
target = dataset[target_feature]
    
cls = svm.SVC(kernel='linear')    
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.7, random_state=209)
cls.fit(X_train, y_train)

pred = cls.predict(X_test)
print("Общая точность: ", metrics.accuracy_score(y_test, y_pred=pred))
print("Precision: ", metrics.precision_score(y_test, y_pred=pred, average='weighted'))
print("Recall: ", metrics.recall_score(y_test, y_pred=pred, average='weighted'))
print("F1-measure: ", metrics.f1_score(y_test, y_pred=pred, average='weighted'))
print("Число опорных векторов: ", len(cls.support_vectors_))
#print(metrics.classification_report(y_test, y_pred=pred))

Общая точность:  0.9442231075697212
Precision:  0.9402390438247012
Recall:  0.9442231075697212
F1-measure:  0.9415670650730412
Число опорных векторов:  22


In [44]:
print(cls.support_)

[  1  30  36  37  42  49  58  72  80  86  93  95  97 105  18  44  54  81
  90   3  28  45]


In [33]:
from sklearn.model_selection import GridSearchCV
 
# defining parameter range
param_grid = {'C': [0.1, 1, 10, 100, 1000],
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf', 'linear', 'poly', 'sigmoid']}
 
grid = GridSearchCV(svm.SVC(), param_grid, refit = True, verbose = 3)

grid.fit(X_train, y_train)
grid_predictions = grid.predict(X_test)

Fitting 5 folds for each of 100 candidates, totalling 500 fits
[CV 1/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.909 total time=   0.0s
[CV 2/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.909 total time=   0.0s
[CV 3/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.952 total time=   0.0s
[CV 4/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.952 total time=   0.0s
[CV 5/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.905 total time=   0.0s
[CV 1/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.955 total time=   0.0s
[CV 2/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.955 total time=   0.0s
[CV 3/5] END .....C=0.1, gamma=1, kernel=linear;, score=1.000 total time=   0.0s
[CV 4/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.952 total time=   0.0s
[CV 5/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.952 total time=   0.0s
[CV 1/5] END .......C=0.1, gamma=1, kernel=poly;, score=0.955 total time=   0.0s
[CV 2/5] END .......C=0.1, gamma=1, kernel=pol

D:\Anaconda\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


[CV 5/5] END ..C=0.1, gamma=0.1, kernel=sigmoid;, score=0.905 total time=   0.0s
[CV 1/5] END .....C=0.1, gamma=0.01, kernel=rbf;, score=0.909 total time=   0.0s
[CV 2/5] END .....C=0.1, gamma=0.01, kernel=rbf;, score=0.909 total time=   0.0s
[CV 3/5] END .....C=0.1, gamma=0.01, kernel=rbf;, score=0.952 total time=   0.0s
[CV 4/5] END .....C=0.1, gamma=0.01, kernel=rbf;, score=0.952 total time=   0.0s
[CV 5/5] END .....C=0.1, gamma=0.01, kernel=rbf;, score=0.905 total time=   0.0s
[CV 1/5] END ..C=0.1, gamma=0.01, kernel=linear;, score=0.955 total time=   0.0s
[CV 2/5] END ..C=0.1, gamma=0.01, kernel=linear;, score=0.955 total time=   0.0s
[CV 3/5] END ..C=0.1, gamma=0.01, kernel=linear;, score=1.000 total time=   0.0s
[CV 4/5] END ..C=0.1, gamma=0.01, kernel=linear;, score=0.952 total time=   0.0s
[CV 5/5] END ..C=0.1, gamma=0.01, kernel=linear;, score=0.952 total time=   0.0s
[CV 1/5] END ....C=0.1, gamma=0.01, kernel=poly;, score=0.909 total time=   0.0s
[CV 2/5] END ....C=0.1, gamm

[CV 2/5] END ......C=1, gamma=0.001, kernel=rbf;, score=0.909 total time=   0.0s
[CV 3/5] END ......C=1, gamma=0.001, kernel=rbf;, score=0.952 total time=   0.0s
[CV 4/5] END ......C=1, gamma=0.001, kernel=rbf;, score=0.952 total time=   0.0s
[CV 5/5] END ......C=1, gamma=0.001, kernel=rbf;, score=0.905 total time=   0.0s
[CV 1/5] END ...C=1, gamma=0.001, kernel=linear;, score=0.955 total time=   0.0s
[CV 2/5] END ...C=1, gamma=0.001, kernel=linear;, score=1.000 total time=   0.0s
[CV 3/5] END ...C=1, gamma=0.001, kernel=linear;, score=1.000 total time=   0.0s
[CV 4/5] END ...C=1, gamma=0.001, kernel=linear;, score=0.952 total time=   0.0s
[CV 5/5] END ...C=1, gamma=0.001, kernel=linear;, score=0.952 total time=   0.0s
[CV 1/5] END .....C=1, gamma=0.001, kernel=poly;, score=0.909 total time=   0.0s
[CV 2/5] END .....C=1, gamma=0.001, kernel=poly;, score=0.909 total time=   0.0s
[CV 3/5] END .....C=1, gamma=0.001, kernel=poly;, score=0.952 total time=   0.0s
[CV 4/5] END .....C=1, gamma

[CV 4/5] END .....C=10, gamma=0.001, kernel=rbf;, score=0.952 total time=   0.0s
[CV 5/5] END .....C=10, gamma=0.001, kernel=rbf;, score=0.905 total time=   0.0s
[CV 1/5] END ..C=10, gamma=0.001, kernel=linear;, score=0.955 total time=   0.0s
[CV 2/5] END ..C=10, gamma=0.001, kernel=linear;, score=1.000 total time=   0.0s
[CV 3/5] END ..C=10, gamma=0.001, kernel=linear;, score=1.000 total time=   0.0s
[CV 4/5] END ..C=10, gamma=0.001, kernel=linear;, score=0.952 total time=   0.0s
[CV 5/5] END ..C=10, gamma=0.001, kernel=linear;, score=0.952 total time=   0.0s
[CV 1/5] END ....C=10, gamma=0.001, kernel=poly;, score=0.909 total time=   0.0s
[CV 2/5] END ....C=10, gamma=0.001, kernel=poly;, score=0.909 total time=   0.0s
[CV 3/5] END ....C=10, gamma=0.001, kernel=poly;, score=0.952 total time=   0.0s
[CV 4/5] END ....C=10, gamma=0.001, kernel=poly;, score=0.952 total time=   0.0s
[CV 5/5] END ....C=10, gamma=0.001, kernel=poly;, score=0.905 total time=   0.0s
[CV 1/5] END .C=10, gamma=0.

[CV 2/5] END C=100, gamma=0.001, kernel=sigmoid;, score=0.955 total time=   0.0s
[CV 3/5] END C=100, gamma=0.001, kernel=sigmoid;, score=0.952 total time=   0.0s
[CV 4/5] END C=100, gamma=0.001, kernel=sigmoid;, score=0.952 total time=   0.0s
[CV 5/5] END C=100, gamma=0.001, kernel=sigmoid;, score=0.810 total time=   0.0s
[CV 1/5] END ...C=100, gamma=0.0001, kernel=rbf;, score=0.955 total time=   0.0s
[CV 2/5] END ...C=100, gamma=0.0001, kernel=rbf;, score=0.955 total time=   0.0s
[CV 3/5] END ...C=100, gamma=0.0001, kernel=rbf;, score=1.000 total time=   0.0s
[CV 4/5] END ...C=100, gamma=0.0001, kernel=rbf;, score=0.952 total time=   0.0s
[CV 5/5] END ...C=100, gamma=0.0001, kernel=rbf;, score=0.905 total time=   0.0s
[CV 1/5] END C=100, gamma=0.0001, kernel=linear;, score=0.955 total time=   0.0s
[CV 2/5] END C=100, gamma=0.0001, kernel=linear;, score=1.000 total time=   0.0s
[CV 3/5] END C=100, gamma=0.0001, kernel=linear;, score=1.000 total time=   0.0s
[CV 4/5] END C=100, gamma=0.

In [34]:
print(grid.best_params_)
print(grid.best_estimator_)

{'C': 0.1, 'gamma': 1, 'kernel': 'poly'}
SVC(C=0.1, gamma=1, kernel='poly')


In [25]:
cancer_data = datasets.load_breast_cancer()

X_train, X_test, y_train, y_test = train_test_split(cancer_data.data, cancer_data.target, test_size=0.4, random_state=209)

cls = svm.SVC(kernel='linear')
cls.fit(X_train, y_train)

pred = cls.predict(X_test)
print("accuracy: ", metrics.accuracy_score(y_test, y_pred=pred))

accuracy:  0.9254385964912281


In [5]:
import umap
reducer = umap.UMAP()

dataset_2d = reducer.fit_transform(dataset_standard)

from sklearn.manifold import TSNE

tsne = TSNE(n_components=2, random_state=0)

dataset_2d = tsne.fit_transform(dataset_standard)
plt.scatter()

569
569
